In [170]:
import pandas as pd
import numpy as np
import numpy.random as random
from route_functions import *
import matplotlib.pyplot as plt

### Variables
We have our system state defined as  - 
* SS: (n, nd, (ch1, st1, r1,m, tbm, sbm), (ch2, st2, r2,m, tb2,m, sb2,m),.. (chn, stn, rn,m, tbn,m, sbn,m))
    * n - total number of available buses
    * m - subscript that denotes route m
    * nd - number of buses deployed
    * chn -  charging state of bus n
    * stn - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * rn,m - the route on which the bus is deployed
    * tbn,m - the previous stop the bus travelled to (travel time)
    * sbn,m - the previous stop which was served by the bus (service time)
        * If tbn= sbn, generate travel time for tbn+1; tbn = tbn+1 if accepted
        * Else if tbn> sbn, generate service time for sbn+1; sbn = sbn+1 if accepted

* t - the clock time/current time
* nrm - number of buses required on route m
* btn - time array for monitoring travel time and service time related to each bus


### Pseudo-code
1. Start at t = 0
2. Generate requirements from route, nd (required number of buses):
    1. Check availability of buses & minimum fuel level/battery capacity required for the routes
    2. Choose number of buses, K, to be deployed based on checks
        1. If K<nd, deploy buses with higher fuel/battery level and recharge/refuel the rest
        2. Else, send buses for recharge/refuel
    3. For each bus k, in {1, 2,.. n}, after selecting a specific route m {stops - 1, 2,... M}:
    4. Initialize j = 0 (Depot_Start)
    5. Generate interarrival times for stop j, update time tbk,j
    6. Generate service time at stop j, update time sbk,j
    7. STOP if j = M; Else update j = j + 1, goto 2.b. 
3. Based on the time we update the System State (SS) and the concerned variables.

### Cases
1. We generate demand for the route at time t,
    Sub-task: Generating series of buses to be deployed
    1. Condition 1: number of buses required <= number of charged/fuelled buses
        1. Deploy buses based on descending order of fuel level/charge
        2. Send buses for refuelling/recharging, if they are below a certain threshold
    2. Condition 2: number of buses required > number of charged/fuelled buses
        1. Send buses for refuelling/recharging
        2. As soon as the bus finishes recharging/refueling, buses get sent to the route
2. Routing of buses:
    1. Route 0: based on type of the bus and refuelling need. Will be a refuelling/ recharging route for the bus
    2. Route 1 to m: Buses will go through the stops till they reach Depot_Stop


## Variables
#### Buses
* bus: ($ch_n, st_n, r_{n,m}, tb_{n,m}, sb_{n,m}$)
    * $ch_n$ - charging state of bus n
    * $st_n$ - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * $r_{n,m}$ - the route on which the bus is deployed
    * $tb_{n,m}$ - the previous stop the bus travelled to (travel time)
    * $sb_{n,m}$ - the previous stop which was served by the bus (service time)
        * If $tb_n$= $sb_n$, generate travel time for $tb_{n+1}$; $tb_n$ = $tb_{n+1}$ if accepted
        * Else if $tb_n$> $sb_n$, generate service time for $sb_{n+1}$; $sb_n$ = $sb_{n+1}$ if accepted
    * Arrival time - route generated arrays
    * Service Time - route generated arrays
    * 

In [171]:
routes = pd.read_excel('Model_Parameters.xlsx', 'Routes')
routes

,recharge_index,recharge_mean,recharge_std,refill_index,refill_mean,refill_std,route_1_index,route_1_mean,route_1_std,route_2_index,route_2_mean,route_2_std
0,1.0,2.0,0.25,1.0,1.5,0.25,1,5.00,1.00,1.0,5.00,1.0
1,0.0,12.0,5.00,0.0,5.0,2.00,0,1.00,0.50,0.0,1.00,0.5
2,1.0,1.0,0.50,1.0,2.0,0.50,1,7.00,2.00,1.0,7.00,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,0,0.75,1.50,0.0,0.75,1.5
4,NaN,NaN,NaN,NaN,NaN,NaN,1,5.00,1.50,1.0,5.00,1.5
5,NaN,NaN,NaN,NaN,NaN,NaN,0,1.20,0.75,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,1,4.00,2.00,NaN,NaN,NaN


In [172]:
#### ASSUMPTION: buses are prioritized based on their index
def next_bus_e(buses):
    min_t = np.inf
    index = None
    for i in range(len(buses)):
        if (buses[i].next_t()<min_t):
            min_t = buses[i].next_t()
            index = i
    return min_t, buses[i].next_e(), index

def available_bus(buses, dem_charge):
    b_charges = [buses[i].charge for i in range(len(buses)) if buses[i].state==-1]
    b_index = [i for i in range(len(buses)) if buses[i].state==-1]
    index = -1
    if len(b_charges)>0:
        if (max(b_charges)>dem_charge):
            for i in range(len(b_charges)):
                if max(b_charges)==b_charges[i]:
                    index = b_index[i]
    return index

def unavailable_bus(buses, min_charge):
    b_charges = [buses[i].charge for i in range(len(buses)) if buses[i].state==-1]
    b_index = [i for i in range(len(buses)) if buses[i].state==-1]
    index = -1
    if (len(b_charges)>0)and(min_charge!=np.inf):
        if (min(b_charges)<min_charge):
            for i in range(len(b_charges)):
                if min(b_charges)==b_charges[i]:
                    index = b_index[i]
    return index

def buses_status(buses):
    n_dep = sum([1 for i in buses if i.state==1])
    n_ref = sum([1 for i in buses if i.state==0])
    n_stds = sum([1 for i in buses if i.state==-1])
    return n_dep, n_ref, n_stds

System State New
SS = (n_buses, curr_dem, buses_deployed, buses_recharge, buses_standstill)
SS = (n_buses, curr_dem, buses_deployed, buses_recharge, buses_standstill, route_1_buses, route_2_buses,....)

#### Monitoring System State 

| Time | System State | Bus | Charge | Route | State | Event | Process Time | Demand-Current | Demand-Actual | Demand-Charge |
| ---- | ------------ | --- | ------ | ----- | ----- | ----- | ------------ | -------------- | ------------- | ------------- |
| 0    | (1,1,1,0,0)  | 1   | 50     | '1'   | 1     | 0     | 0            | 0              | 0             | 15            |
| 10.3 | (1,0,1,0,0)  | 1   | 48     | '1'   | 1     | 1     | 10.3         | -              | -             | -             |
| 12.3 | (1,0,1,0,0)  | 1   | 47.5   | '1'   | 1     | 0     | 2.0          | -              | -             | -             |

___

#### Monitoring Bus Deployments

| Time | Demand-Current | Demand-Actual | Demand-Charge | Bus | Charge | Route | Event Array  | Time Array   |
| ---- | -------------- | ------------- | ------------- | --- | ------ | ----- | ------------ | ------------ |
| 0    | 0              | 0             | 15            | 1   | 50     | '1'   | [1, 0, 1]    | [12, 35, 41] |
| 40   | 41             | 40            | 20            | 1   | 35     | '2'   | [1, 0, 1]    | [48, 50, 55] |



In [173]:
SS_cols = ['Time', 'System_State', 'Bus', 'Charge', 'Route', 
           'State', 'Event', 'Process_Time', 'Demand_Current', 
           'Demand_Actual', 'Demand_Charge']
BD_cols = ['Time', 'Demand_Current', 'Demand_Actual', 'Demand_Charge', 
           'Bus', 'Charge', 'Route', 'Event_Array', 'Time_Array']

def SS_update(SS_table, t, buses, dct, bus_e, t_updt, index=-1, dem_ct=0, dem_at=0, dem_c=0):
    ss_d = {}
    ss_d['Time'] = t_updt
    ss_arr = [len(buses), dct]
    ss_dep, ss_re, ss_ss = buses_status(buses)
    ss_arr.append(ss_dep)
    ss_arr.append(ss_re)
    ss_arr.append(ss_ss)
    ss_d['System_State'] = ss_arr
    if index!=-1:
        ss_d['Bus'] = index + 1
        ss_d['Charge'] = buses[index].charge
        ss_d['Route'] = buses[index].route
        ss_d['State'] = buses[index].state
    else:
        ss_d['Bus'] = np.nan
        ss_d['Charge'] = np.nan
        ss_d['Route'] = None
        ss_d['State'] = np.nan
    ss_d['Event'] = bus_e
    ss_d['Process_Time'] = t_updt - t    
    if dem_c==0:
        ss_d['Demand_Current'] = np.nan
        ss_d['Demand_Actual'] = np.nan
        ss_d['Demand_Charge'] = np.nan
    else:
        ss_d['Demand_Current'] = dem_ct
        ss_d['Demand_Actual'] = dem_at
        ss_d['Demand_Charge'] = dem_c
    SS_table = SS_table.append(ss_d, ignore_index=True)
    return SS_table

def BD_update(BD_table, t_updt, dem_ct, dem_at, dem_c, buses, index):
    bd_d = {}
    bd_d['Time'] = t_updt
    bd_d['Demand_Current'] = dem_ct
    bd_d['Demand_Actual'] = dem_at
    bd_d['Demand_Charge'] = dem_c
    bd_d['Bus'] = index + 1
    bd_d['Charge'] = buses[index].charge
    bd_d['Route'] = buses[index].route
    bd_d['State'] = buses[index].state
    bd_d['Event_Array'] = np.array(buses[index].event_arr)
    bd_d['Time_Array'] = np.array(buses[index].time_arr)
    BD_table = BD_table.append(bd_d, ignore_index=True)
    return BD_table

In [197]:
# customizable parameters
spare_buses = 2
average_bus_speed = 10
running_consumption = 0.5             # fuel(lts.)/kWh per minute
service_consumption = 0.5
refuel_stations = 2
refuel_consumption = -10

# tank level/charge of buses
level_mean = 85       # normal distribution.. can be changed
level_std = 2.5
refuel = 'refill'

# setting up initial values for simulation
n_buses = 3                                # number of buses
n_routes = 2                               # number of routes
buses = [bus(level_mean, level_std) for i in range(n_buses)]    # fleet of buses
t = 0                                      # start time of simulation
T = 60

# updating demands
demand_at, demand_r, demand_c = gen_demands(n_routes, T)
demand_ct = demand_at.copy()         # demand current time and demand actual time
dct_flag = 0                  # demand at current time

ss_table = pd.DataFrame(columns=SS_cols)
bd_table = pd.DataFrame(columns=BD_cols)

'''# -----
#### creating fake data & demands
t = 0                                      # start time of simulation
T = 60
buses = [bus(5, 5) for i in range(1)]
demand_at = [0, 40, np.inf]
demand_r = ['1', '2', None]
demand_c = [15, 20, np.inf]
demand_ct = demand_at.copy()         # demand current time and demand actual time
dct_flag = 0                  # demand at current time'''


# -----
# Initial SS update
time_check = np.inf
ss_table = SS_update(ss_table, t, buses, dct_flag, np.nan, np.nan)

# Simulate! Simulate! Simulate!        
while (t<T)or(time_check!=np.inf):
    print('\n---\nNew Loop')
    
    time_check = min(next_bus_e(buses)[0], demand_ct[0])
    print('Time check -', time_check)
    
    # -----
    # Priority One: Updating demands
    if ((demand_ct[0]!=np.inf) and
        (demand_ct[0]==time_check)):
        new_demand = [1 for i in range(len(demand_ct)) if demand_ct[i]==time_check]
        dct_flag = sum(new_demand)
        
    print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
    print('\tdct_flag -', dct_flag)
    
    
    #----------
    # Main Simulation
    bus_chk = available_bus(buses, demand_c[0])
    print('Bus Check -', bus_chk)
    
    
    #-----
    # Case 1: Sending low-fuel buses to refuel
    refuel_index = unavailable_bus(buses, min(demand_c))
    if (#(t<T) and
        (refuel_index!=-1) and
        (buses_status(buses)[1]<refuel_stations)): 
        # or((dct_flag>0)and(refuel_index!=np.inf)):- might create complications
        
        print('Sending Bus for Refuel')
        buses[refuel_index].assign_route(routes, refuel, t)
        bus_e = 0
        t_updt = t
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, refuel_index)
        bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, refuel_index)
        
        t = t_updt
        refuel_index = -1
    
    
    #-----
    # Case 2: Checking bus availability and deploying buses
    elif ((t<T) and
          (demand_ct[0]==time_check) and
          (demand_ct[0]!=np.inf) and
          (bus_chk!=-1)):
        
        print('Deploying Bus')
        dem_ct, dem_at = demand_ct.pop(0), demand_at.pop(0)
        dem_c, dem_r = demand_c.pop(0), demand_r.pop(0)
        index = available_bus(buses, dem_c)
        buses[index].assign_route(routes, dem_r, t)
        t_updt = dem_ct
        bus_e = 0
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index, dem_ct, dem_at, dem_c)
        bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, index)
        print(buses[index].event_arr, buses[index].time_arr)
        dct_flag -= 1
        t = t_updt
        
        dem_ct, dem_at, dem_c, dem_r = np.nan, np.nan, np.nan, np.nan
        t_updt = np.nan
        index = np.nan
        
    
    #-----
    # Case 3: Checking next bus event and updating SS
    elif (((t<T) and
          (next_bus_e(buses)[0]==time_check) and 
          (next_bus_e(buses)[0]!=np.inf) and 
          ((buses_status(buses)[0]>0) or (buses_status(buses)[1]>0))) or
          ((t<T) and
          (demand_ct[0]==time_check) and 
          (demand_ct[0]!=np.inf) and 
          (bus_chk==-1))):
        
        print('Updating next bus event')
        index = next_bus_e(buses)[2]
        t_updt = buses[index].time_arr.pop(0)
        bus_e = buses[index].event_arr.pop(0)
        
        if (bus_e==0)and(buses[index].state==1):
            mul_c = service_consumption
        elif (bus_e==1):
            mul_c = running_consumption
        elif (bus_e==0)and(buses[index].state==0):
            mul_c = refuel_consumption
        buses[index].charge = buses[index].charge - (t_updt - t)*mul_c
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index)
        t = t_updt
        
        # if its the last event for the bus, update bus parameters to standstill
        if buses[index].next_e()==np.inf:
            buses[index].state = -1
            buses[index].route = None
            buses[index].time_arr = list()
            buses[index].event_arr = list()
        index = np.nan
        t_updt = np.nan
        bus_e = np.nan
        
        
    #-----
    # Case 4: All demands are completed within the timeframe 
    elif ((t<T) and
          (time_check==np.inf)):
        
        print('Jumping to EOD')
        t_updt = T
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt)
        t = t_updt    
    
    
    #-----
    # Case 5: Demands still exist beyond timeframe and need to be met
    elif ((t>T) and
          (demand_ct[0]==time_check) and
          (demand_ct[0]!=np.inf) and
          (bus_chk!=-1)):
        
        print('Deploying Bus beyond timeframe')
        dem_ct, dem_at = demand_ct.pop(0), demand_at.pop(0)
        dem_c, dem_r = demand_c.pop(0), demand_r.pop(0)
        index = available_bus(buses, dem_c)
        buses[index].assign_route(routes, dem_r, t)
        t_updt = dem_ct
        bus_e = 0
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index, dem_ct, dem_at, dem_c)
        bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, index)
        dct_flag -= 1
        t = t_updt
        
        dem_ct, dem_at, dem_c, dem_r = np.nan, np.nan, np.nan, np.nan
        t_updt = np.nan
        index = np.nan
        

    #-----
    # Case 6: Going beyond timeframe, checking deployed buses and updating SS
    elif (((t>T) and
          (next_bus_e(buses)[0]==time_check) and
          (next_bus_e(buses)[0]!=np.inf)) or
          ((t>T) and
          (demand_ct[0]==time_check) and 
          (demand_ct[0]!=np.inf) and 
          (bus_chk==-1))):
        print('Updating next bus event beyond timeframe')
        index = next_bus_e(buses)[2]
        t_updt = buses[index].time_arr.pop(0)
        bus_e = buses[index].event_arr.pop(0)
        
        if (bus_e==0)and(buses[index].state==1):
            mul_c = service_consumption
        elif (bus_e==1):
            mul_c = running_consumption
        elif (bus_e==0)and(buses[index].state==0):
            mul_c = refuel_consumption
        buses[index].charge = buses[index].charge - (t_updt - t)*mul_c
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index)
        t = t_updt
        
        # if its the last event for the bus, update bus parameters to standstill
        if buses[index].next_e()==np.inf:
            buses[index].state = -1
            buses[index].route = None
            buses[index].time_arr = list()
            buses[index].event_arr = list()
        index = np.nan
        t_updt = np.nan
        bus_e = np.nan
    
    #-----
    # Case 7: All demands are completed within the timeframe 
    elif ((t>T) and
          (time_check==np.inf)):
        
        print('Jumping to EOD')
        t_updt = np.inf        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt)
        t = t_updt    
    
    
    
    print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
    print('\tdct_flag -', dct_flag)
    
    # updating the current demand - if demand exists it'll automatically be updated
    for k in range(dct_flag):
        demand_ct[k] = t
        
    print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
    print('\tdct_flag -', dct_flag)
    print('t -', t)


---
New Loop
Time check - 0.0
	Demands - [0.0, 0.0, 20.0, 30.0, 40.0, inf] [0.0, 0.0, 20.0, 30.0, 40.0, inf] ['1', '2', '1', '2', '1', None] [25.0, 20.0, 25.0, 20.0, 25.0, inf]
	dct_flag - 2
Bus Check - 2
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [5.866, 6.133, 28.0874, 30.0629, 36.7665, 37.1693, 50.0442]
	Demands - [0.0, 20.0, 30.0, 40.0, inf] [0.0, 20.0, 30.0, 40.0, inf] ['2', '1', '2', '1', None] [20.0, 25.0, 20.0, 25.0, inf]
	dct_flag - 1
	Demands - [0.0, 20.0, 30.0, 40.0, inf] [0.0, 20.0, 30.0, 40.0, inf] ['2', '1', '2', '1', None] [20.0, 25.0, 20.0, 25.0, inf]
	dct_flag - 1
t - 0.0

---
New Loop
Time check - 0.0
	Demands - [0.0, 20.0, 30.0, 40.0, inf] [0.0, 20.0, 30.0, 40.0, inf] ['2', '1', '2', '1', None] [20.0, 25.0, 20.0, 25.0, inf]
	dct_flag - 1
Bus Check - 0
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [4.0707, 4.5748, 15.8599, 15.8659, 21.8831]
	Demands - [20.0, 30.0, 40.0, inf] [20.0, 30.0, 40.0, inf] ['1', '2', '1', None] [25.0, 20.0, 25.0, inf]
	dct_flag - 0
	Demands - [20.0, 30.0

C:\Users\krishrao\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [198]:
ss_table

,Time,System_State,Bus,Charge,Route,State,Event,Process_Time,Demand_Current,Demand_Actual,Demand_Charge
0,NaN,"[3, 0, 0, 0, 3]",NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0000,"[3, 2, 1, 0, 2]",3,89.782464,1,1,0,0.0000,0.0000,0.0,25.0
2,0.0000,"[3, 1, 2, 0, 1]",1,86.905442,2,1,0,0.0000,0.0000,0.0,20.0
3,4.0707,"[3, 0, 2, 0, 1]",1,84.870092,2,1,1.0,4.0707,NaN,NaN,NaN
4,4.5748,"[3, 0, 2, 0, 1]",1,84.618042,2,1,0.0,0.5041,NaN,NaN,NaN
5,5.8660,"[3, 0, 2, 0, 1]",3,89.136864,1,1,1,1.2912,NaN,NaN,NaN
6,6.1330,"[3, 0, 2, 0, 1]",3,89.003364,1,1,0,0.2670,NaN,NaN,NaN
7,15.8599,"[3, 0, 2, 0, 1]",1,79.754592,2,1,1.0,9.7269,NaN,NaN,NaN
8,15.8659,"[3, 0, 2, 0, 1]",1,79.751592,2,1,0.0,0.0060,NaN,NaN,NaN
9,20.0000,"[3, 1, 3, 0, 0]",2,85.171465,1,1,0,4.1341,20.0000,20.0,25.0


In [199]:
bd_table

,Time,Demand_Current,Demand_Actual,Demand_Charge,Bus,Charge,Route,Event_Array,Time_Array,State
0,0.0000,0.0000,0.0,25.0,3,89.782464,1,"[1, 0, 1, 0, 1, 0, 1]","[5.866, 6.133, 28.0874, 30.0629, 36.7665, 37.1...",1.0
1,0.0000,0.0000,0.0,20.0,1,86.905442,2,"[1.0, 0.0, 1.0, 0.0, 1.0]","[4.0707, 4.5748, 15.8599, 15.8659, 21.8831]",1.0
2,20.0000,20.0000,20.0,25.0,2,85.171465,1,"[1, 0, 1, 0, 1, 0, 1]","[23.3398, 23.5494, 34.5824, 34.5958, 40.9624, ...",1.0
3,30.0000,30.0000,30.0,20.0,1,78.810042,2,"[1.0, 0.0, 1.0, 0.0, 1.0]","[32.0044, 32.5756, 44.1621, 44.4593, 52.0039]",1.0
4,46.4903,46.4903,40.0,25.0,2,79.540315,1,"[1, 0, 1, 0, 1, 0, 1]","[51.0461, 51.267, 73.032, 73.9874, 77.0563, 77...",1.0


In [181]:
bd_table['Time_Array'][4]

array([35.5478, 35.9076, 48.2595, 51.2171, 61.0015, 61.1395, 74.4042])

___
## Testing Area

In [112]:
demand_at, demand_r, demand_c = gen_demands(n_routes, T)
demand_at

[0.0, 0.0, 20.0, 30.0, 40.0, inf]

In [118]:
demand_at.pop(0)
demand_at

[]

In [ ]:
# testing for demand creation and updation

#demand = 0
arr = [0, 1, 0, 1, 2, 3]
test = 5
new_demand = [1 for i in range(len(arr)) if arr[i]==test]
demand = demand + sum(new_demand)
demand

In [ ]:
routes[routes.columns[routes.columns.str.match('route_1')]]

In [ ]:
print(route_time(routes, 'recharge', 0))
print(route_time(routes, 'refill', 0))
print(route_time(routes, '2', 0))

In [ ]:
n_buses = 4
buses = [bus(50, 5) for i in range(n_buses)]
t = 10
buses[0].assign_route(routes, 'refill', t)
buses[1].assign_route(routes, '1', t)
buses[2].assign_route(routes, '1', t)
print(buses[0].time_arr)
print(buses[0].event_arr)
buses_status(buses)

In [ ]:
print(buses[0].next_t())
print(buses[0].next_e())
print(buses[0].time_arr.pop(0))
print(buses[0].event_arr.pop(0))
print(buses[0].next_t())
print(buses[0].next_e())

In [ ]:
#### ASSUMPTION: buses are prioritized based on their index

def next_bus_e(buses):
    min_t = np.inf
    index = None
    for i in range(len(buses)):
        print(buses[i].next_t())
        if (buses[i].next_t()<min_t):
            min_t = buses[i].next_t()
            index = i
    return min_t, buses[i].next_e(), index

# testing 
buses = [bus(50, 5) for i in range(3)]
buses[0].assign_route(routes, '1', 0)
buses[1].assign_route(routes, '1', 0)
buses[2].assign_route(routes, '1', 0)
print(buses[0].time_arr)
print(buses[0].event_arr)
print(buses[1].time_arr)
print(buses[1].event_arr)
print(buses[2].time_arr)
print(buses[2].event_arr)
next_bus_e(buses)

def available_bus(buses, dem_charge):
    print(buses)
    b_charges = [buses[i].charge for i in range(len(buses)) if buses[i].state==-1]
    b_index = [i for i in range(len(buses)) if buses[i].state==-1]
    index = np.nan
    if len(b_charges)>0:
        if (max(b_charges)>dem_charge):
            for i in range(len(b_charges)):
                if max(b_charges)==b_charges[i]:
                    index = b_index[i]
    return index

check = bus(20, 4)
buses = [bus(25, 5) for i in range(1)]
buses[0].state = 1
available_bus(buses, 20)

#### Testing for route demands

In [ ]:
# route demand functions
array_t = np.array(range(0, 12*60+1, 60))

a, b, c, d = 2, 3, 0, 75
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='r', label='bus_1', marker='o')

a, b, c, d = 2, 3, -180, 120
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='b', label='bus_2', marker='o')

a, b, c, d = 1, 3, 0, 45
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='g', label='bus_3', marker='o')

plt.xlabel('Time')
plt.ylabel('Buses')
plt.legend()
plt.show()